In [5]:
#using h5py to unpack the archived file which holds the information for our 
import numpy as np
import h5py

In [6]:
#let's extract our training, test and validation data
#method seen in [1] p21
with h5py.File('../HDF5/hdf5_uTHCD_compressed.h5', 'r') as hdf:
    base_items = list(hdf.items())
    print(f'Items in base dir: {base_items}')
    G1 = hdf.get('Train Data')
    G1_items = list(G1.items())
    print(f'items in group 1: {G1_items}')
    
    G2 = hdf.get('Test Data')
    G2_items = list(G2.items())
    print(f'items in group 2: {G2_items}')
    
    x_train = np.array(G1.get('x_train'))
    y_train = np.array(G1.get('y_train'))
    
    x_test = np.array(G2.get('x_test'))
    y_test = np.array(G2.get('y_test'))
    
    x_val = x_train[-7870:,:,:]
    y_val = y_train[-7870:]
    
    x_train = x_train[:-7870,:,:]
    y_train = y_train[:-7870]
    
x_train.shape

Items in base dir: [('Test Data', <HDF5 group "/Test Data" (2 members)>), ('Train Data', <HDF5 group "/Train Data" (2 members)>)]
items in group 1: [('x_train', <HDF5 dataset "x_train": shape (62870, 64, 64), type "|u1">), ('y_train', <HDF5 dataset "y_train": shape (62870,), type "<i8">)]
items in group 2: [('x_test', <HDF5 dataset "x_test": shape (28080, 64, 64), type "|u1">), ('y_test', <HDF5 dataset "y_test": shape (28080,), type "<i8">)]


(55000, 64, 64)

Now that we have the data, we actually need to use it.

In [7]:
import scipy
import pandas as pd
from sklearn import svm

In [8]:
#let's try a SVC

In [9]:
x_val.shape

(7870, 64, 64)

In [10]:
#It's important to note here we are using very basic parameters and the simple linear kernel
svc = svm.SVC(gamma=0.001,kernel='linear', C = 100)

#reshaping the data to work for SVC while maintaining images. Likely poor optimization
x_train_svc = x_train.reshape(55000,-1)
x_test_svc = x_test.reshape(28080, -1)
x_val_svc = x_val.reshape(7870,-1)
x_val.shape

(7870, 64, 64)

In [11]:
#we're gonna train on only 1000 samples until we figure out complexity issues
SAMPLE_SIZE = 1000
svc.fit(x_train_svc[:SAMPLE_SIZE],y_train[:SAMPLE_SIZE])

SVC(C=100, gamma=0.001, kernel='linear')

In [12]:
predictions = svc.predict(x_test_svc[:SAMPLE_SIZE])
print("end")
(predictions, y_test[:SAMPLE_SIZE])

correct = 0
incorrect = 0
for pred, expect in zip(predictions,y_test[:SAMPLE_SIZE]):
    if pred == expect:
        correct += 1
    else:
        incorrect += 1


end


In [13]:
#some analysis of our model's performance with the linear kernel
#accuracy:
correct/SAMPLE_SIZE

0.448

In [14]:
#now let's see how it does on the validation data
predictions = svc.predict(x_val_svc[:SAMPLE_SIZE])
correct = 0
incorrect = 0
for pred, expect in zip(predictions,y_val[:SAMPLE_SIZE]):
    if pred == expect:
        correct += 1
    else:
        incorrect += 1


In [15]:
correct/SAMPLE_SIZE

0.518

Likely slightly more accurate due to the validation data being closer in size to SAMPLE_SIZE than testing data

In [16]:
import matplotlib as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from keras import backend as K

In [17]:
#helper functions to compute f1 score later
#taken from [3]
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [18]:
#set up and build the initial model of 2D convolutional layers and MaxPooling
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(64,64,1)))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(32, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))


#will now flatten conv layers and add 3 dense layers of size 1024 and 512
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(156)) #156 distinct char classes per [1] p.3
model.summary() #to output a depiction of the network

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 31, 31, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_3 (Dense)             (None, 1024)             

In [23]:
#compile the model using SparseCategoricalCrossentropy (see [2]) 
#here we only use 4 epochs to train
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy',f1_m,precision_m,recall_m]) 
res = model.fit(x_train,y_train, epochs=5, validation_data=(x_test,y_test)) #fit the training and test data to the model

Epoch 1/5


2023-11-12 10:42:13.402094: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 225280000 exceeds 10% of free system memory.


1719/1719 [==============================] - 233s 135ms/step - loss: 0.1905 - accuracy: 0.9434 - f1_m: 1.3109 - precision_m: 1.0722 - recall_m: 1.7390 - val_loss: 0.7892 - val_accuracy: 0.8201 - val_f1_m: 1.3469 - val_precision_m: 1.1062 - val_recall_m: 1.7363
Epoch 2/5
1719/1719 [==============================] - 232s 135ms/step - loss: 0.1299 - accuracy: 0.9617 - f1_m: 1.2234 - precision_m: 1.0559 - recall_m: 1.4640 - val_loss: 0.7960 - val_accuracy: 0.8220 - val_f1_m: 1.2124 - val_precision_m: 1.0927 - val_recall_m: 1.3711
Epoch 3/5
1719/1719 [==============================] - 233s 135ms/step - loss: 0.1100 - accuracy: 0.9687 - f1_m: 1.1735 - precision_m: 1.0477 - recall_m: 1.3409 - val_loss: 0.9093 - val_accuracy: 0.8282 - val_f1_m: 1.2075 - val_precision_m: 1.0681 - val_recall_m: 1.4012
Epoch 4/5
1719/1719 [==============================] - 232s 135ms/step - loss: 0.0972 - accuracy: 0.9715 - f1_m: 1.1495 - precision_m: 1.0391 - recall_m: 1.2927 - val_loss: 1.0390 - val_accuracy: 0

In [24]:
test_loss, test_acc, test_f1, test_precision, test_recall = model.evaluate(x_test, y_test)
print(f'loss: {test_loss}\nacc: {test_acc}\nf1: {test_f1}\nprecision: {test_precision}\nrecall: {test_recall}')

878/878 [==============================] - 18s 21ms/step - loss: 1.1357 - accuracy: 0.8031 - f1_m: 1.1621 - precision_m: 1.0709 - recall_m: 1.2790
loss: 1.1357309818267822
acc: 0.8030982613563538
f1: 1.1620622873306274
precision: 1.0709322690963745
recall: 1.2789952754974365


In [25]:
val_loss, val_acc, val_f1, val_precision, val_recall = model.evaluate(x_val,y_val)
print(f'loss: {val_loss}\nacc: {val_acc}\nf1: {val_f1}\nprecision: {val_precision}\nrecall: {val_recall}')

246/246 [==============================] - 5s 21ms/step - loss: 0.7097 - accuracy: 0.8709 - f1_m: 1.1531 - precision_m: 1.0556 - recall_m: 1.2780
loss: 0.7097020149230957
acc: 0.8709021806716919
f1: 1.1530578136444092
precision: 1.0556143522262573
recall: 1.278039574623108


REFERENCES:
[1]: N. Shaffi and F. Hajamohideen, "uTHCD: A New Benchmarking for Tamil Handwritten OCR," in IEEE Access, vol. 9, pp. 101469-101493, 2021, doi: 10.1109/ACCESS.2021.3096823.
[2]: https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy 
'